In [40]:
import torch
from torch import nn as nn
import torch.optim as optim
import numpy as np
import random
import torch.nn.functional as F
#from utils import ngsimDataset,maskedNLL,maskedMSE,maskedNLLTest
import time
import math
from __future__ import print_function, division
from torch.utils.data import Dataset, DataLoader
import scipy.io as scp

from tqdm import tqdm




In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [46]:
def prepare_data(batch_size, t_h): #假设 t_h = 8
    inputs = torch.randn(batch_size, t_h, 2) # len(features) = 14
    return inputs    # batch_size, seq_length, features

## Load Data

In [43]:
class lstm_encoder(nn.Module):

    def __init__(self, input_size, hidden_size=128, num_layers = 1):
        
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.embedding = nn.Linear(input_size, 64)
        self.hidden_size = 128
        self.num_layers = 1

        # define LSTM layer
        self.lstm = nn.LSTM(64, hidden_size = 128,  # input 是 (batch_size, sequence_size, input_size)
                            num_layers = 1, batch_first=False)       

        #define activation:
        self.leaky_relu = torch.nn.LeakyReLU(0.1)
        
    def forward(self, x_input):
        
        embedded = self.embedding(x_input)
        embedded = self.leaky_relu(embedded)
        lstm_out, self.hidden = self.lstm(embedded)
        
        
        return lstm_out, self.hidden     
    
 

In [47]:
torch.manual_seed(1729)
inputs = prepare_data(20, 8)
input_sizes = 2
encoder = lstm_encoder(input_sizes)

In [49]:
def prepare_data(batch_size, t_h): #假设 t_h = 8
    inputs = torch.randn(t_h,batch_size, 2) # len(features) = 2
    return inputs    # seq_length,  batch_size, features

torch.manual_seed(1729)
inputs = prepare_data(20, 8)
input_sizes = 2
encoder = lstm_encoder(input_sizes)

output, hidden = encoder(inputs)  
print(inputs.shape)
print(output.shape) # output: [time, batch, features]
print(hidden[0].shape)
print(hidden[1].shape)

torch.Size([8, 20, 2])
torch.Size([8, 20, 128])
torch.Size([1, 20, 128])
torch.Size([1, 20, 128])


从Encoder到decoder的维度拼接：

In [6]:
def one_hot_coding(batch_size):
    m_class = torch.zeros(6,6)
    for i in range(0,6):
        m_class[i][i] = 1
    one_hot = torch.empty(6,batch_size,6)
    for i in range(6):
        temp = m_class[i].view(1,-1)
        for j in range(0, batch_size-1):
            
            temp = torch.cat((temp,m_class[i].view(1,-1)))
        one_hot[i] = temp
    return one_hot




In [20]:
def concat_output(output, one_hot,batch_size):
    dec_input = torch.empty(6,batch_size,134)
    for i in range(6):
        dec_input[i] = torch.cat((output[0], one_hot[i]),1)
        
    return dec_input.view(dec_input.shape[0],1,batch_size,-1)


In [36]:
class lstm_decoder(nn.Module):
    ''' Decodes hidden state output by encoder '''
    
    def __init__(self, input_size, output_size, hidden_size=128, num_layers = 1):

        
        super(lstm_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = 128
        self.num_layers = 1
        #self.embedding = nn.Linear(input_size, 64)
        self.lstm = nn.LSTM(134, hidden_size = self.hidden_size,
                            num_layers = self.num_layers, batch_first=False)
        self.linear = nn.Linear(self.hidden_size, output_size)           

    def forward(self, encoder_hidden_cat, hidden):
        
        lstm_out, self.hidden = self.lstm(encoder_hidden_cat, hidden)   # use encoder's final hidden state concatted with one-hot as input
        output = self.linear(lstm_out)     
        return output, self.hidden

    def init_hidden(self, batch_size=1):
        
        # (self.n_layers, batch_size, self.hidden_dim)
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size)) 
        
    

In [37]:
class lstm_seq2seq(nn.Module):
    ''' train LSTM encoder-decoder'''
    
    def __init__(self, enc_input_size, hidden_size=128, output_timestep = 25, output_size = 2):

        super(lstm_seq2seq, self).__init__()

        self.enc_input_size = enc_input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.encoder = lstm_encoder(self.enc_input_size)
        self.decoder = lstm_decoder(input_size=134, output_size=2)   # outputsize is tricky
        #self.encoder_hidden = self.encoder.init_hidden(batch_size)
    
    def forward(self, hist, nbr, target_batch, loss_fn, output_timestep=25, batch_size=1):
        #how to concatenate hist and nbr to input_batch: concat on third dim, and padding
        input_batch = hist          # 只尝试本车
        
        encoder_output, encoder_hidden = self.encoder(input_batch)
        # use concatted encoder_hidden_states as decoder's input 
        
        batch_loss = 0.
        decoder_hidden = self.decoder.init_hidden(batch_size)
        one_hot = one_hot_coding(batch_size)
        decoder_input_all = concat_output(encoder_output, one_hot, batch_size=1)  
        for i in range(0,6):
            pred = torch.empty(25,batch_size, 2)
            decoder_input = decoder_input_all[i]
            for j in range(output_timestep):
            #decoder outputs 
                temp, decoder_output_hidden = self.decoder.forward(decoder_input, decoder_hidden)    # decoder的输入是encoder的hidden_state 与 one_hot concat在一起
                pred[j] = temp.squeeze()
                decoder_hidden = decoder_output_hidden                                                       
            
            #computing loss
            loss = loss_fn(pred, target_batch)   # pred:[25,batch_size,2], target:[25,batch_size,2]  # how to compute NLLloss?
            batch_loss += loss

        return batch_loss, pred
    

In [ ]:
decoder = lstm_decoder(input_size=128, output_size=2)
decoder_output, decoder_hidden = decoder(output, cat_hidden_states)
print(decoder_output.shape)

NameError: name 'output' is not defined

# training

In [10]:
def valid(model, valid_loader):
    model.eval()
    valid_loss = 0.
    for data in tqdm(valid_loader):
        hist, nbrs, _, _, _, fut, _ = data
        hist,nbrs,fut = hist.to(device), nbrs.to(device), fut.to(device)
        loss,_ = model(hist, nbrs, fut)
        valid_loss += loss.item()
    return valid_loss / len(valid_loader)

In [29]:
def train_model(data_dir, n_epochs=100, _batch_size=1, _lr = 0.001, load_model=False, model_path='output/model0.pth'):

    trSet = ngsimDataset(data_dir+'TrainSet.mat')
    valSet = ngsimDataset(data_dir+'ValSet.mat')
    trDataloader = DataLoader(trSet,batch_size=_batch_size,shuffle=True,collate_fn=trSet.collate_fn)
    valDataloader = DataLoader(valSet,batch_size=_batch_size,shuffle=True,collate_fn=valSet.collate_fn)
    # training
    model = lstm_seq2seq(2).to(device) # 目前尝试只考虑本车：input_size=2

    if load_model == True:
        ckpt = torch.load(model_path)
        model.load_state_dict(ckpt['state_dict'])

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=_lr)
        
    min_loss = 100.  # why?

    for epoch in range(n_epochs):
        total_loss = 0.
        for data in tqdm(trDataloader):
            hist, nbrs, _, _, _, fut, _ = data
            hist,nbrs,fut = hist.to(device), nbrs.to(device), fut.to(device)
            loss,_ = model(hist, nbrs, fut, loss_fn, output_timestep=25, batch_size=1)

            # zero the gradient
            optimizer.zero_grad()
                          
            # backpropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        epoch_loss = total_loss/len(trDataloader)
        print('epoch:{},avg_loss:{}'.format(epoch, epoch_loss))
        #save best model
        if epoch_loss < min_loss:
            min_loss = epoch_loss
            torch.save({'epoch': epoch, 'state_dict': model.state_dict()},
                           'output/model_{}.pth'.format(epoch))
            print("epoch:%d Model Saved" % epoch)

        #validation
        if epoch % 10 == 0:
            valid_loss = valid(model, valDataloader)
            print('validation epoch:{},valid_loss:{}'.format(epoch, valid_loss))
            model.train()


In [39]:
data_dir='/mnt/e/Northwestern University/Courses/2022winter/pattern recognition/projects/data/'
train_model(data_dir, n_epochs=100, _batch_size=1, _lr = 0.001, load_model=False, model_path='output/model0.pth')

  0%|          | 2431/5922867 [04:34<185:30:14,  8.87it/s]


KeyboardInterrupt: 

# Predict

In [ ]:
def test_model(data_dir, _batch_size=16, model_path = 'output/model0.pth'):
    testSet = ngsimDataset(data_dir+'TestSet.mat')
    test_loader = DataLoader(testSet,batch_size=_batch_size,shuffle=False,collate_fn=testSet.collate_fn)
    
    model = lstm_seq2seq(14,128).to(device)
    ckpt = torch.load(model_path)
    model.load_state_dict(ckpt['state_dict'])
    model.eval()

    for data in tqdm(test_loader):
        hist, nbrs, _, _, _, fut, _ = data
        hist,nbrs,fut = hist.to(device), nbrs.to(device), fut.to(device)
        _,pred = model(hist, nbrs, fut)

        #multiply with manuver probability



In [3]:
class ngsimDataset(Dataset):


    def __init__(self, mat_file, t_h=30, t_f=50, d_s=2, enc_size = 64, grid_size = (13,3)):
        self.D = scp.loadmat(mat_file)['traj']
        self.T = scp.loadmat(mat_file)['tracks']
        self.t_h = t_h  # length of track history
        self.t_f = t_f  # length of predicted trajectory
        self.d_s = d_s  # down sampling rate of all sequences
        self.enc_size = enc_size # size of encoder LSTM
        self.grid_size = grid_size # size of social context grid



    def __len__(self):
        return len(self.D)



    def __getitem__(self, idx):

        dsId = self.D[idx, 0].astype(int)
        vehId = self.D[idx, 1].astype(int)
        t = self.D[idx, 2]
        grid = self.D[idx,8:]
        neighbors = []

        # Get track history 'hist' = ndarray, and future track 'fut' = ndarray
        hist = self.getHistory(vehId,t,vehId,dsId)
        fut = self.getFuture(vehId,t,dsId)

        # Get track histories of all neighbours 'neighbors' = [ndarray,[],ndarray,ndarray]
        for i in grid:
            neighbors.append(self.getHistory(i.astype(int), t,vehId,dsId))

        # Maneuvers 'lon_enc' = one-hot vector, 'lat_enc = one-hot vector
        lon_enc = np.zeros([2])
        lon_enc[int(self.D[idx, 7] - 1)] = 1
        lat_enc = np.zeros([3])
        lat_enc[int(self.D[idx, 6] - 1)] = 1

        return hist,fut,neighbors,lat_enc,lon_enc



    ## Helper function to get track history
    def getHistory(self,vehId,t,refVehId,dsId):
        if vehId == 0:
            return np.empty([0,2])
        else:
            if self.T.shape[1]<=vehId-1:
                return np.empty([0,2])
            refTrack = self.T[dsId-1][refVehId-1].transpose()
            vehTrack = self.T[dsId-1][vehId-1].transpose()
            refPos = refTrack[np.where(refTrack[:,0]==t)][0,1:3]

            if vehTrack.size==0 or np.argwhere(vehTrack[:, 0] == t).size==0:
                 return np.empty([0,2])
            else:
                stpt = np.maximum(0, np.argwhere(vehTrack[:, 0] == t).item() - self.t_h)
                enpt = np.argwhere(vehTrack[:, 0] == t).item() + 1
                hist = vehTrack[stpt:enpt:self.d_s,1:3]-refPos

            if len(hist) < self.t_h//self.d_s + 1:
                return np.empty([0,2])
            return hist



    ## Helper function to get track future
    def getFuture(self, vehId, t,dsId):
        vehTrack = self.T[dsId-1][vehId-1].transpose()
        refPos = vehTrack[np.where(vehTrack[:, 0] == t)][0, 1:3]
        stpt = np.argwhere(vehTrack[:, 0] == t).item() + self.d_s
        enpt = np.minimum(len(vehTrack), np.argwhere(vehTrack[:, 0] == t).item() + self.t_f + 1)
        fut = vehTrack[stpt:enpt:self.d_s,1:3]-refPos
        return fut



    ## Collate function for dataloader
    def collate_fn(self, samples):

        # Initialize neighbors and neighbors length batches:
        nbr_batch_size = 0
        for _,_,nbrs,_,_ in samples:
            nbr_batch_size += sum([len(nbrs[i])!=0 for i in range(len(nbrs))])
        maxlen = self.t_h//self.d_s + 1
        nbrs_batch = torch.zeros(maxlen,nbr_batch_size,2)


        # Initialize social mask batch:
        pos = [0, 0]
        mask_batch = torch.zeros(len(samples), self.grid_size[1],self.grid_size[0],self.enc_size)
        mask_batch = mask_batch.byte()


        # Initialize history, history lengths, future, output mask, lateral maneuver and longitudinal maneuver batches:
        hist_batch = torch.zeros(maxlen,len(samples),2)
        fut_batch = torch.zeros(self.t_f//self.d_s,len(samples),2)
        op_mask_batch = torch.zeros(self.t_f//self.d_s,len(samples),2)
        lat_enc_batch = torch.zeros(len(samples),3)
        lon_enc_batch = torch.zeros(len(samples), 2)


        count = 0
        for sampleId,(hist, fut, nbrs, lat_enc, lon_enc) in enumerate(samples):

            # Set up history, future, lateral maneuver and longitudinal maneuver batches:
            hist_batch[0:len(hist),sampleId,0] = torch.from_numpy(hist[:, 0])
            hist_batch[0:len(hist), sampleId, 1] = torch.from_numpy(hist[:, 1])
            fut_batch[0:len(fut), sampleId, 0] = torch.from_numpy(fut[:, 0])
            fut_batch[0:len(fut), sampleId, 1] = torch.from_numpy(fut[:, 1])
            op_mask_batch[0:len(fut),sampleId,:] = 1
            lat_enc_batch[sampleId,:] = torch.from_numpy(lat_enc)
            lon_enc_batch[sampleId, :] = torch.from_numpy(lon_enc)

            # Set up neighbor, neighbor sequence length, and mask batches:
            for id,nbr in enumerate(nbrs):
                if len(nbr)!=0:
                    nbrs_batch[0:len(nbr),count,0] = torch.from_numpy(nbr[:, 0])
                    nbrs_batch[0:len(nbr), count, 1] = torch.from_numpy(nbr[:, 1])
                    pos[0] = id % self.grid_size[0]
                    pos[1] = id // self.grid_size[0]
                    mask_batch[sampleId,pos[1],pos[0],:] = torch.ones(self.enc_size).byte()
                    count+=1

        return hist_batch, nbrs_batch, mask_batch, lat_enc_batch, lon_enc_batch, fut_batch, op_mask_batch

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=08756b88-eec3-4b1b-a0c9-04763ec70cac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>